In [ ]:
import os
os.environ['PATH_TO_REPO'] = '/Users/stevie/repos/lingo_kit_data'

In [ ]:
repo_path = os.environ['PATH_TO_REPO']
assert(os.path.exists(repo_path))

In [ ]:
import pandas as pd
import sys
sys.path.append(repo_path)
from utils.csv_helper import get_all_csv_files_rec
import yaml

In [ ]:
files = get_all_csv_files_rec(os.path.join(repo_path, 'dataframes/dataframes_by_pos/'))

In [ ]:
path = 'dataframe_management/base_terms_to_topics.yaml'
topic_obj = yaml.safe_load(open(os.path.join(repo_path, path), 'r'))

In [ ]:
print(f'{str(topic_obj['base_terms']['senza']).replace("'", '').replace("[", "").replace("]", "")}')

In [ ]:
missing = []
for f in files:
    df = pd.read_csv(f)
    row = df.iloc[0]
    italian_term = row['base_lemma_italian']
    if italian_term not in topic_obj['base_terms']:
        missing.append(italian_term)
    topics_str = f'{str(topic_obj['base_terms'][italian_term]).replace("'", '').replace("[", "").replace("]", "").replace(', ', ',')}'
    df['topics'] = topics_str
    df.to_csv(f, index=False)
    # print(f'Wrote {f}')
    # break
print(missing)

In [ ]:
path = '/Users/stevie/repos/lingo_kit_data/dataframes/old_dataframes/spotify_lessons.csv'
df = pd.read_csv(path)
df.columns

In [ ]:
terms = []
for t in df['italian_term'].unique().tolist():
    terms += [t.lower().replace('?', '').replace('!', '').replace('.', '').replace(',', '').strip() for t in t.split(' ')]
terms = list(set([x for x in terms if len(x) > 0 and x != '/']))

In [ ]:
found_terms = set()
for f in files:
    df = pd.read_csv(f)

    found_spotify = False
    for i, row in df.iterrows():
        t = row['term_italian'].lower().strip()
        if t in terms:
            found_spotify = True
            found_terms.add(t)

    if found_spotify:
        for i, row in df.iterrows():
            tags = row['topics']
            if 'Spotify Lessons' not in tags:
                tags = tags + ',Spotify Lessons'
            df.loc[i, 'topics'] = tags

    df.to_csv(f, index=False)
not_found_terms = set(terms) - found_terms
not_found_terms